In [1]:
import pandas as pd 
import numpy as np
import os 
from sklearn.metrics import roc_auc_score

In [2]:
train = {}
for f in os.listdir("../output/train/"):
    print(f)
    train[f] = pd.read_csv("../output/train/"+f)

cnn_2d_conv.csv
cnn_bi_lstm.csv
cnn_capsule.csv
cnn_conv1D_num.csv
NN_GRU_pooling.csv
train_oof_lgbm.csv
train_oof_lr.csv
train_oof_nb.csv
train_oof_wb.csv


In [3]:
columns = ['id', 'toxic_oof', 'severe_toxic_oof', 'obscene_oof', 'threat_oof', 'insult_oof', 'identity_hate_oof']
train['train_oof_lgbm.csv'] = train['train_oof_lgbm.csv'][columns]
train['train_oof_wb.csv'] = train['train_oof_wb.csv'][columns]

In [4]:
for name, df in train.items():
    if df.shape[0] != 159571: print("test: %s" %name)
    if df.shape[1] != 7: print("more columns: %s" %name)

In [5]:
test = {}
for f in os.listdir("../output/test/"):
    print(f)
    test[f] = pd.read_csv("../output/test/"+f)

cnn_2d_conv.csv
cnn_bi_lstm.csv
cnn_capsule.csv
cnn_conv1D_num.csv
NN_GRU_pooling.csv
test_oof_lgbm.csv
test_oof_lr.csv
test_oof_nb.csv
test_oof_wb.csv


In [6]:
for name, df in test.items():
#     if df.shape[0] != 159571: print("test: %s" %name)
    if df.shape[1] != 7: print("more columns: %s" %name)

In [7]:
# test['NN_GRU_pooling.csv'].iloc[:, 1:]
idx = test['NN_GRU_pooling.csv']['id']
for k in test.keys():
    print(np.sum(test[k]['id'] != idx))

0
0
0
0
0
0
0
0
0


In [8]:
idx.head(10) == train['NN_GRU_pooling.csv']['id'].head(10)

0    True
1    True
2    True
3    True
4    True
5    True
6    True
7    True
8    True
9    True
Name: id, dtype: bool

In [9]:
# # test['NN_GRU_pooling.csv'].iloc[:, 1:]
# for k in train.keys():
#     print(np.sum(train[k]['id'] != idx))

In [10]:
train_input = pd.read_csv("../input/train.csv")

In [11]:
arr = np.zeros((9, train_input.shape[0], 6))
i = 0
for k in train.keys():
    arr[i,:,:] = train[k].iloc[:, 1:]
    i = i + 1

In [12]:
corr_train = np.zeros((9, 9, 6))
for c in range(6):
    for i in range(9):
        for j in range(i+1, 9):
            corr_train[i,j,c] = np.corrcoef(x=arr[i,:,c], y=arr[j,:,c])[0,1]

In [13]:
target = train_input.iloc[:, 2:].values

In [14]:
scores = [roc_auc_score(target, arr[i,:,:]) for i in range(9)]

In [15]:
list(zip(scores, train.keys()))

[(0.98709519515642175, 'cnn_2d_conv.csv'),
 (0.98690860151520166, 'cnn_bi_lstm.csv'),
 (0.98515369775656481, 'cnn_capsule.csv'),
 (0.98519169849234611, 'cnn_conv1D_num.csv'),
 (0.98596544883295978, 'NN_GRU_pooling.csv'),
 (0.97854726461641839, 'train_oof_lgbm.csv'),
 (0.98611238536201207, 'train_oof_lr.csv'),
 (0.98258157084229814, 'train_oof_nb.csv'),
 (0.9824335204725213, 'train_oof_wb.csv')]

BLENDING

In [16]:
# average
roc_auc_score(target, arr.sum(axis=0)/9)

0.99094097708271311

In [17]:
# average + rank
def np_rank(array, prob=True):
    """ ranking 2D array by columns """
    assert len(array.shape) == 2, "wrong shape"
    ranks = np.empty_like(array)
    for i in np.arange(array.shape[1]):
        temp = array[:, i].argsort()
        ranks[temp, i] = np.arange(len(array))
    if prob: return ranks/array.shape[0]
    else: return ranks
    
arr_ranked = np.zeros_like(arr)
for i in range(9):
    arr_ranked[i,:,:] = np_rank(arr[i,:,:], False)
arr_ranked += 1

In [18]:
roc_auc_score(target, arr_ranked.sum(axis=0)/9)

0.98995797025260679

In [19]:
corr_train_ranked = np.zeros((9, 9, 6))
for c in range(6):
    for i in range(9):
        for j in range(i+1, 9):
            corr_train_ranked[i,j,c] = np.corrcoef(x=arr_ranked[i,:,c], y=arr_ranked[j,:,c])[0,1]

In [20]:
pd.DataFrame(corr_train_ranked[:,:,3])

,0,1,2,3,4,5,6,7,8
0,0.0,0.713418,0.490468,0.771218,0.715602,0.271843,0.640883,0.566598,0.228384
1,0.0,0.000000,0.530570,0.744964,0.699499,0.272909,0.601381,0.534581,0.202075
2,0.0,0.000000,0.000000,0.511163,0.467078,0.286517,0.418464,0.383094,0.172018
3,0.0,0.000000,0.000000,0.000000,0.721090,0.236588,0.586024,0.539777,0.207494
4,0.0,0.000000,0.000000,0.000000,0.000000,0.248195,0.574379,0.575046,0.242599
5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.337988,0.271887,0.148414
6,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.647188,0.391408
7,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.397262
8,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [21]:
## HMEAN

from scipy.stats import hmean
# hmean(arr_ranked, axis=0)

In [22]:
hmean_proba = np.power(np.power(arr_ranked, -1).sum(axis=0), -1)
hmean_proba *= 9
hmean_proba /= hmean_proba.max(0)

In [23]:
hmean_proba

array([[ 0.21996686,  0.17960523,  0.08205559,  0.01923763,  0.07164118,
         0.23414688],
       [ 0.36330324,  0.29849675,  0.21959823,  0.24741893,  0.34398114,
         0.063213  ],
       [ 0.5296202 ,  0.13279321,  0.23750357,  0.17211049,  0.40544446,
         0.06720217],
       ..., 
       [ 0.70388144,  0.03904969,  0.58640402,  0.44345498,  0.63789527,
         0.61407199],
       [ 0.46637805,  0.16225647,  0.3137666 ,  0.40242958,  0.4239803 ,
         0.17412582],
       [ 0.72694848,  0.54641106,  0.69332158,  0.66725651,  0.65647486,
         0.47529344]])

In [24]:
roc_auc_score(target, hmean_proba)

0.98829456026896312

OPTIMAL WEIGHTS

In [219]:
from scipy.optimize import minimize

In [249]:
def cost_function(w):
    w = np.array(w).reshape(1,9)
    return 1 - roc_auc_score(target, np.tensordot(w, arr, axes=([1],[0]))[0]/np.sum(w))

In [251]:
w0 = [1,1,1,1,1,1,1,1,1]
cost_function(w0)

0.0090590229172868852

In [285]:
w_opt = res.copy()

In [301]:
w_opt

{'final_simplex': (array([[ 1.13850466,  1.63472416,  0.86600171,  0.41580508,  1.40662179,
           1.37803494,  0.28409879,  1.22928972,  0.61248468],
         [ 1.1385048 ,  1.63472393,  0.86600162,  0.41580523,  1.40662175,
           1.378035  ,  0.28409886,  1.22928969,  0.61248459],
         [ 1.1385047 ,  1.63472413,  0.86600177,  0.41580496,  1.40662175,
           1.37803506,  0.28409882,  1.22928971,  0.61248469],
         [ 1.13850481,  1.63472402,  0.86600169,  0.41580516,  1.40662178,
           1.37803501,  0.28409879,  1.22928972,  0.61248455],
         [ 1.13850465,  1.63472408,  0.8660018 ,  0.41580512,  1.40662182,
           1.3780348 ,  0.28409882,  1.22928975,  0.61248465],
         [ 1.13850471,  1.634724  ,  0.86600177,  0.4158049 ,  1.40662181,
           1.37803509,  0.28409881,  1.22928972,  0.61248473],
         [ 1.1385046 ,  1.63472433,  0.86600192,  0.41580474,  1.40662164,
           1.37803512,  0.28409881,  1.2292897 ,  0.61248483],
         [ 1.1385

In [322]:
def cost_function(w, info):
    
    w = np.array(w).reshape(1,9)
    score = 1 - roc_auc_score(target, np.tensordot(w, arr_ranked, axes=([1],[0]))[0]/np.sum(w))
    
    # verbose
    if info['itera'] % 50 == 0: 
        print(info['itera'], ["%.2f"%i for i in w[0]], score)
    info['itera'] += 1
    
    return score

w0 = [1,1,1,1,1,1,1,1,1]
res = minimize(cost_function, w0, method='Nelder-Mead', tol=1e-6, args=({'itera':1},),
               options={'maxiter': 2000, 'disp': True})

50 ['1.40', '1.51', '1.19', '0.32', '1.07', '0.83', '1.20', '0.80', '0.87'] 0.00997509203253
100 ['1.76', '1.83', '0.99', '0.61', '0.84', '0.79', '1.29', '0.38', '0.77'] 0.00994658595017
150 ['1.65', '1.96', '0.97', '0.52', '0.72', '0.85', '1.20', '0.44', '0.83'] 0.00993943613635
200 ['1.58', '2.03', '0.90', '0.52', '0.64', '0.91', '1.09', '0.53', '0.90'] 0.00993804993586
250 ['1.62', '2.11', '0.81', '0.57', '0.69', '0.89', '1.12', '0.49', '0.92'] 0.00993654102988
300 ['1.60', '2.11', '0.80', '0.59', '0.67', '0.90', '1.13', '0.49', '0.93'] 0.00993619803503
350 ['1.60', '2.12', '0.81', '0.59', '0.66', '0.90', '1.13', '0.49', '0.93'] 0.00993618458245
400 ['1.60', '2.12', '0.80', '0.59', '0.66', '0.90', '1.13', '0.49', '0.93'] 0.00993612634749
450 ['1.60', '2.12', '0.80', '0.59', '0.66', '0.90', '1.13', '0.49', '0.93'] 0.00993610995204
500 ['1.60', '2.12', '0.80', '0.59', '0.66', '0.90', '1.13', '0.49', '0.93'] 0.00993609932943
550 ['1.60', '2.12', '0.80', '0.59', '0.66', '0.90', '1.13', 

In [323]:
from scipy.optimize import differential_evolution

In [324]:
bounds = [(0.2, 4),(0.2, 4),(0.2, 4),(0.2, 4),(0.2, 4),(0.2, 4),(0.2, 4),(0.2, 4),(0.2, 4)]
res1 = differential_evolution(cost_function, bounds, popsize=20, maxiter=30,
                              tol=1e-6, args=({'itera':1},), disp=True)

50 ['3.78', '3.29', '1.26', '2.64', '2.89', '2.92', '2.92', '1.23', '3.64'] 0.0100391843027
100 ['3.73', '1.26', '2.19', '3.93', '1.04', '0.76', '2.38', '1.64', '0.32'] 0.0102681810555
150 ['2.00', '3.35', '0.61', '2.25', '3.71', '2.19', '3.79', '3.71', '3.06'] 0.0101714833392
200 ['0.43', '3.44', '3.16', '1.86', '2.02', '2.74', '2.86', '0.94', '3.85'] 0.0101095266174
250 ['0.67', '3.46', '2.66', '1.16', '3.61', '1.54', '3.81', '1.27', '1.01'] 0.0100872930783
300 ['3.20', '3.18', '1.44', '2.82', '2.66', '0.25', '3.84', '2.85', '3.88'] 0.0101757208629
350 ['0.69', '0.59', '1.87', '3.46', '2.25', '2.89', '2.14', '1.04', '2.31'] 0.0102794104563
differential_evolution step 1: f(x)= 0.0099688
400 ['1.05', '2.57', '0.59', '0.44', '0.45', '2.29', '3.03', '1.66', '1.92'] 0.0102542148293
450 ['1.88', '3.79', '2.40', '2.75', '1.90', '1.07', '1.30', '1.51', '2.17'] 0.0100294496204
500 ['3.29', '3.42', '1.15', '0.65', '1.45', '3.35', '3.21', '2.14', '0.68'] 0.0101085610807
differential_evolution s

3900 ['3.34', '3.85', '1.52', '0.97', '1.51', '1.58', '2.14', '0.59', '1.63'] 0.00994323108381
3950 ['3.06', '4.00', '1.49', '0.74', '0.58', '1.52', '1.76', '0.82', '2.02'] 0.00995215411871
differential_evolution step 21: f(x)= 0.00993626
4000 ['2.76', '3.99', '1.32', '1.06', '1.02', '1.51', '2.32', '0.88', '1.65'] 0.00993688498265
4050 ['2.90', '4.00', '1.60', '1.09', '1.26', '1.48', '2.31', '1.07', '1.75'] 0.00993686409966
4100 ['3.23', '3.98', '1.70', '0.80', '1.36', '1.75', '2.56', '0.96', '1.82'] 0.00993918726968
differential_evolution step 22: f(x)= 0.00993626
4150 ['3.24', '3.91', '1.47', '1.02', '1.28', '1.53', '2.19', '0.86', '1.80'] 0.00993666964174
4200 ['2.78', '3.94', '1.40', '0.80', '1.15', '1.52', '2.39', '0.77', '1.86'] 0.00994105202998
4250 ['2.89', '3.85', '1.38', '0.85', '0.70', '1.22', '2.41', '0.77', '1.74'] 0.00994536632645
4300 ['2.82', '3.84', '1.40', '0.81', '1.14', '1.44', '2.07', '0.65', '1.75'] 0.00993956816011
differential_evolution step 23: f(x)= 0.0099362

In [317]:
w_opt_GP = (res1.x/np.sum(res1.x)).reshape(1,9)

In [311]:
w_opt['x']/np.sum(w_opt['x'])

array([ 0.12698637,  0.18233364,  0.09659198,  0.04637801,  0.15689159,
        0.15370307,  0.03168777,  0.13711235,  0.06831523])

In [320]:
roc_auc_score(target, np.tensordot(w_opt_GP, arr, axes=([1],[0]))[0]/np.sum(w_opt_GP))

0.99124259558884609

In [31]:
w_opt_GP = np.array([ 0.08503522,  0.18484535,  0.09647718,  0.05563641,  0.16151241,
         0.1702125 ,  0.03064141,  0.14804717,  0.06759235]).reshape(1,9)

SAVE RESULTS

In [33]:
test_input = pd.read_csv("../input/test.csv")

In [34]:
test_input.shape

(153164, 2)

In [35]:
arr_test = np.zeros((9, test_input.shape[0], 6))
i = 0
for k in test.keys():
    arr_test[i,:,:] = test[k].iloc[:, 1:]
    i = i + 1

In [36]:
sample = pd.read_csv("../input/sample_submission.csv")

In [37]:
opt_submission = pd.DataFrame(np.tensordot(w_opt_GP, arr_test, axes=([1],[0]))[0]/np.sum(w_opt_GP))
opt_submission = pd.concat((test_input['id'], opt_submission), axis=1)
opt_submission.columns = sample.columns
opt_submission.to_csv("../output/blending/optW.csv", index=False)

In [38]:
av_submission = pd.DataFrame(arr_test.sum(axis=0)/9)
av_submission = pd.concat((test_input['id'], av_submission), axis=1)
av_submission.columns = sample.columns
av_submission.to_csv("../output/blending/avgW.csv", index=False)

In [52]:
arr_test = np.zeros((9, test_input.shape[0], 6))
i = 0
for k in test.keys():
    arr_test[i,:,:] = np_rank(test[k].iloc[:, 1:].values, True)
    i = i + 1

In [55]:
rk_submission = pd.DataFrame(arr_test.sum(axis=0)/9)
rk_submission = pd.concat((test_input['id'], rk_submission), axis=1)
rk_submission.columns = sample.columns
rk_submission.to_csv("../output/blending/rnkW.csv", index=False)

BLEND OF BLENDS

In [39]:
blend1=pd.read_csv("../input/superblend/blend1.csv")
blend2=pd.read_csv("../input/superblend/blend2.csv")
blend3=pd.read_csv("../input/superblend/blend3.csv")
blend4=pd.read_csv("../input/superblend/blend4.csv")

In [57]:
superblend = np.zeros((5,test_input.shape[0],6))

In [67]:
superblend[0] = rk_submission.iloc[:, 1:].values
superblend[1] = blend1.iloc[:, 1:].values
superblend[2] = blend2.iloc[:, 1:].values
superblend[3] = blend3.iloc[:, 1:].values
superblend[4] = blend4.iloc[:, 1:].values

In [86]:
corr_blend = np.zeros((5, 5, 6))
for c in range(6):
    for i in range(5):
        for j in range(i, 5):
            corr_blend[i,j,c] = np.corrcoef(x=superblend[i,:,c], y=superblend[j,:,c])[0,1]

In [100]:
pd.DataFrame(corr_blend[:,:,0])

,0,1,2,3,4
0,1.000000,0.830901,0.824838,0.834558,0.835831
1,0.830901,1.000000,0.982932,0.999133,0.992297
2,0.824838,0.982932,1.000000,0.981015,0.973974
3,0.834558,0.999133,0.981015,1.000000,0.994329
4,0.835831,0.992297,0.973974,0.994329,1.000000


In [101]:
pd.DataFrame(corr_blend[:,:,1])

,0,1,2,3,4
0,1.000000,0.508474,0.511552,0.511300,0.469774
1,0.508474,1.000000,0.965304,0.997388,0.980920
2,0.511552,0.965304,1.000000,0.960895,0.946994
3,0.511300,0.997388,0.960895,1.000000,0.984140
4,0.469774,0.980920,0.946994,0.984140,1.000000


In [102]:
pd.DataFrame(corr_blend[:,:,2])

,0,1,2,3,4
0,1.000000,0.731200,0.732318,0.731820,0.713681
1,0.731200,1.000000,0.977757,0.999091,0.992963
2,0.732318,0.977757,1.000000,0.976096,0.969600
3,0.731820,0.999091,0.976096,1.000000,0.994966
4,0.713681,0.992963,0.969600,0.994966,1.000000


In [105]:
pd.DataFrame(corr_blend[:,:,4])

,0,1,2,3,4
0,1.000000,0.732958,0.721968,0.731780,0.713678
1,0.732958,1.000000,0.979652,0.998679,0.989320
2,0.721968,0.979652,1.000000,0.977866,0.968254
3,0.731780,0.998679,0.977866,1.000000,0.992495
4,0.713678,0.989320,0.968254,0.992495,1.000000


In [110]:
sp_blend = pd.DataFrame(superblend.sum(0)/5)
sp_blend = pd.concat((test_input['id'], sp_blend), axis=1)
sp_blend.columns = sample.columns
sp_blend.to_csv("../output/blending/super1.csv", index=False)

BLEND OF BLENDS 2

In [120]:
superblend2 = np.zeros((5,test_input.shape[0],6))
superblend2[0] = rk_submission.iloc[:, 1:].values
superblend2[1] = np_rank(blend1.iloc[:, 1:].values)
superblend2[2] = np_rank(blend2.iloc[:, 1:].values)
superblend2[3] = np_rank(blend3.iloc[:, 1:].values)
superblend2[4] = np_rank(blend4.iloc[:, 1:].values)

In [121]:
corr_blend2 = np.zeros((5, 5, 6))
for c in range(6):
    for i in range(5):
        for j in range(i, 5):
            corr_blend2[i,j,c] = np.corrcoef(x=superblend2[i,:,c], y=superblend2[j,:,c])[0,1]

In [132]:
sp_blend2 = pd.DataFrame(superblend2.sum(0)/5)
sp_blend2 = pd.concat((test_input['id'], sp_blend2), axis=1)
sp_blend2.columns = sample.columns
sp_blend2.to_csv("../output/blending/super2.csv", index=False)

In [133]:
sp_blend.head(10)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.962443,0.504755,0.946246,0.269728,0.906679,0.504141
1,0000247867823ef7,0.065706,0.081662,0.057533,0.047989,0.073377,0.073182
2,00013b17ad220c46,0.103721,0.122895,0.120845,0.105508,0.099039,0.109934
3,00017563c3f7919a,0.043648,0.079742,0.051865,0.110356,0.056725,0.043524
4,00017695ad8997eb,0.125544,0.111034,0.110081,0.111592,0.102759,0.093299
5,0001ea8717f6de06,0.070736,0.091131,0.061811,0.128756,0.097955,0.073337
6,00024115d4cbde0f,0.085209,0.058355,0.082334,0.071435,0.099183,0.071008
7,000247e83dcc1211,0.565829,0.139256,0.175499,0.174975,0.227719,0.136222
8,00025358d4737918,0.179779,0.089423,0.137929,0.084212,0.160088,0.108597
9,00026d1092fe71cc,0.043974,0.050013,0.040931,0.071251,0.049970,0.038933


In [134]:
sp_blend2.head(10)

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.970052,0.982741,0.973948,0.984079,0.984544,0.983074
1,0000247867823ef7,0.218147,0.426647,0.166599,0.232809,0.305230,0.450070
2,00013b17ad220c46,0.400633,0.558264,0.440877,0.481117,0.344947,0.407380
3,00017563c3f7919a,0.110667,0.356066,0.110225,0.497501,0.157950,0.127627
4,00017695ad8997eb,0.474516,0.341802,0.420991,0.350849,0.404653,0.302251
5,0001ea8717f6de06,0.240492,0.367507,0.177702,0.646288,0.425573,0.309933
6,00024115d4cbde0f,0.320815,0.226271,0.303174,0.206919,0.376886,0.282419
7,000247e83dcc1211,0.790939,0.575002,0.719562,0.804242,0.759611,0.650604
8,00025358d4737918,0.632018,0.272495,0.665583,0.300327,0.683666,0.540829
9,00026d1092fe71cc,0.114540,0.153399,0.156732,0.219509,0.246100,0.066333
